# reading data from dataset

In [76]:
#  read files from cnn and dailymail and add them to a dictionary where key is file name and values are list of 
#  sentences in that file.
from os import listdir,getcwd
from os.path import isfile, join
def readCorpus(datasetDirectoryName):
    docSummaryDict = {}
    docSentencesDict = {}
    summaryline = False
    # datasetDirectoryName = ['cnn', 'dailymail']
    for dirName in datasetDirectoryName:
        filepath = join(getcwd(), dirName,'stories')
        allFiles = [f for f in listdir(filepath) if not (f.startswith('.')) and isfile(join(filepath, f))]
        for file in allFiles:
            filename = join(filepath, file)
            if 'cnn' in filename:
                fileKey = 'cnn_' + file
            else:
                fileKey = 'dailymail_' + file
            for line in open(filename,'r'): 
                if line != '\n':
                    if '@highlight' in line:
                        summaryline = True
                        continue
                    if not summaryline:
                        if fileKey in docSentencesDict:
                            docSentencesDict[fileKey].append(line.strip())
                        else:
                            docSentencesDict[fileKey] = [line.strip()]
                    else:
                        summaryline = False
                        if fileKey in docSummaryDict:
                            docSummaryDict[fileKey].append(line.strip())
                        else:
                            docSummaryDict[fileKey] = [line.strip()]
    return docSentencesDict, docSummaryDict 

# Term Doc Frequency dictionary

In [148]:
#  funciton to get term-document frequency
#  docSentencesDict should be stemmed words dictionary
#  returns you count of words appear in how many documents for idf
from collections import Counter
from nltk import word_tokenize
def getTermDocfrequency(docSentencesDict):
    termDocFreqDict = Counter()
#     each k is a unique file and v is list of sentences
    for k in docSentencesDict:
        v = docSentencesDict[k]
        fileSet = None
        for line in v:
            lineSet = {word for word in word_tokenize(line)}
#             print(lineSet)
            if fileSet is None:
                fileSet = lineSet
            else:
                fileSet = fileSet | lineSet
        for word in fileSet:
            termDocFreqDict[word] += 1
    return termDocFreqDict        

# remove stop words

In [31]:
# function to read stop words from a text file
def readStopWords(filename):
    text = []
    if(filename.endswith('txt')):
        file = open(filename, 'r')
        for line in file:
            text.append(line.strip())
        return text
    else:
        return None;


from nltk import word_tokenize
def removeStopWords(docSentencesDict):
    returnDocSentencesDict = {}
#     returnText = ''
 #  list of stopWords ( removed following words from list -> eight, eleven, fifteen, first, five, forty, four, nine, 
#   one, six, sixty, twelve, twenty, two, ten, )
# http://xpo6.com/download-stop-word-list/
    stopWordsFile = 'stop-word-list.txt'
    stopWords = readStopWords(stopWordsFile)
    if stopWords is None:
        raise Exception('Couldn\'t parse the given file. Stop Words list is empty. Please provide a text file to parse.')
#     print(stopWords)
    for file in docSentencesDict:
        modifiedTextArr = []
        textArr = docSentencesDict[file]
        for text in textArr:
            modifiedText = ''
            words = word_tokenize(text)
            for word in words:
                if word not in stopWords:
                    modifiedText += word + ' '
            modifiedTextArr.append(modifiedText)
        returnDocSentencesDict[file] = modifiedTextArr
    return returnDocSentencesDict     

In [2]:
# print(removeStopWords('This is four times better than previous solution'))

This four times better previous solution 


# Stemming

In [33]:
# nltk.download()
# creating stemmed text and stemmedTextDict(for term frequency)
from nltk.stem import PorterStemmer
from nltk import word_tokenize
def getStemmedText(docSentencesDict):
    returnedDocSentencesDict = {}
    stemmedWordFrequencyDict = {}
    ps = PorterStemmer()
    for file in docSentencesDict:
        modifiedTextArr = []
        textArr = docSentencesDict[file]
        for text in textArr:
            stemmedText = ''
#         text = docSentencesDict[file]
        
            words = word_tokenize(text)
            for word in words:
                stemmedWord = ps.stem(word)
                stemmedText += stemmedWord + ' '
                if(stemmedWordFrequencyDict.get(stemmedWord, None) == None):
                    stemmedWordFrequencyDict[stemmedWord] =1
                else:
                    stemmedWordFrequencyDict[stemmedWord] += 1 
            modifiedTextArr.append(stemmedText)
        returnedDocSentencesDict[file] = modifiedTextArr
    return returnedDocSentencesDict, stemmedWordFrequencyDict

In [5]:
# print(getStemmedText('python pythonly pythoning pythoned jatin sankar. jatin Sankar python'))

# Data set downloaded using following link
# https://cs.nyu.edu/~kcho/DMQA/

# Computing Lex Rank Scores

In [166]:
from nltk import word_tokenize
from collections import Counter
from math import pow, log, sqrt
from nltk import word_tokenize
def idfModifiedCosine(sentence1,sentence2, totalNumberOfDocs, termDocfreqDict):
#     print('sentence1 is ' + sentence1)
#     print('sentence2 is ' + sentence2)
    sent1TFDict = Counter()
    sent2TFDict = Counter()
    idfDict = {}
    wordsInbothSentences = None
    sentence1Arr = word_tokenize(sentence1)
    sentence2Arr = word_tokenize(sentence2)
    for word in sentence1Arr:
        sent1TFDict[word] += 1
        if wordsInbothSentences is None:
            wordsInbothSentences = {word}
        else:
            wordsInbothSentences = wordsInbothSentences | {word}
        if word not in idfDict:
#             print('calculate idf for word : ' +  word + ' ' +  str(termDocfreqDict[word]))
            idfDict[word] = log(totalNumberOfDocs / termDocfreqDict[word])
    for word in sentence2Arr:
        sent2TFDict[word] += 1
        if wordsInbothSentences is None:
            wordsInbothSentences = {word}
        else:
            wordsInbothSentences = wordsInbothSentences | {word}
        if word not in idfDict:
#             print('calculate idf for word : ' +  word + ' ' +  str(termDocfreqDict[word]))
            idfDict[word] = log(totalNumberOfDocs / termDocfreqDict[word])
    num  = 0
    denSent1 = 0
    denSent2 = 0
    for word in wordsInbothSentences:
        num += sent1TFDict.get(word,0) * sent2TFDict.get(word,0) * pow(idfDict[word],2)
        denSent1 += pow(sent1TFDict.get(word,0) * idfDict[word],2)
        denSent2 += pow(sent2TFDict.get(word,0) * idfDict[word],2)
#     print('denSent1 is ' + str(denSent1))
#     print('denSent2 is ' + str(denSent2))
    return num / (sqrt(denSent1) * sqrt(denSent2))

In [7]:
#  implementing matrix Product
import numpy as np
def matrixProduct(matA, matB):
    return np.matmul(matA, matB)

In [8]:
import numpy as np
def matrixTranspose(mat):
    return np.transpose(mat)

In [170]:
import numpy as np
def matrixDifference(mat1,mat2):
    return np.subtract(mat1,mat2)

In [10]:
from math import pow
# http://www.personal.soton.ac.uk/jav/soton/HELM/workbooks/workbook_30/30_4_matrx_norms.pdf
def calculateEucledeanNorm(mat):
    val = 0
    for row in mat:
        for col in range(len(row)):
            val += pow(row[col],2)
    return pow(val,1/2)

In [173]:
#  implementing power method
def powerMethod(cosineMatrix, N, tolerance):
    cosineMatrixTranspose = matrixTranspose(cosineMatrix)
    initializeP = [[(1/N) for x in range(N)] for y in range(N)]
    t = 0
    currentP = None
    while True:
        if currentP is None:
            oldP = initializeP
        else:
            oldP = currentP
        t += 1
        currentP = matrixProduct(cosineMatrixTranspose,oldP)
        differenceMatrix = matrixDifference(currentP, oldP)
        difference = calculateEucledeanNorm(differenceMatrix)      
        if difference < tolerance:
            return currentP      

In [7]:
# # split Text into sentences
# def SplitTextIntoSentences(text):
#     updatedText = ''
#     for i in range(1,len(text)):
#         if text[i-1] == '.' and text[i] == ' ':
#             updatedText += '\n'
#         else:
#             updatedText += text[i]
#     sentences = updatedText.split('\n')
#     return sentences

In [94]:
#  Lex Rank
#  need to check from where do we get tolerance, threshold
#  idf-modified-cosine
#  We might get divide by zero error, need to think how to avoid that.

def lexRank(sentences,threshold, tolerance, totalNumberOfDocs, termDocfreqDict):
    n = len(sentences)
    cosineMatrix = [[0 for x in range(n)] for y in range(n)]
    degree = [1 for x in range(n)]
    for i in range(0,n):
        for j in range(0,n):
            cosineMatrix[i][j] = idfModifiedCosine(sentences[i],sentences[j], totalNumberOfDocs, termDocfreqDict)
            if cosineMatrix[i][j] > threshold:
                cosineMatrix[i][j] = 1
                degree[i] += 1
            else:
                cosineMatrix[i][j] = 0
    for i in range(0,n):
        for j in range(0,n):
            cosineMatrix[i][j] = cosineMatrix[i][j]/degree[i]
    L = powerMethod(cosineMatrix, n, tolerance)
    return L

# Commands to run Lex Rank

In [ ]:
# docSentencesDict = readCorpus(['Test_cnn', 'Test_dailymail'])
# #  preprocessing
# removedStopWordsDocSentencesDict = removeStopWords(docSentencesDict)
# # stemmedWordFrequencyDict is not need in our implementation
# stemmedDocSentencesDict, stemmedWordFrequencyDict = getStemmedText(removedStopWordsDocSentencesDict)
# termDocFrequencyDict = getTermDocfrequency(stemmedDocSentencesDict)
# totalDocs = len(docSentencesDict)
# # testFile will be array of stemmed sentences in a test file
# # testFile = 
#  threshold is in range of [0.1 - 0.3]
#  very high thresholds may lose almost all of the information in a similarity matrix
# threshold = 0.1
#  tolerance is also consider as damping factor, range [0.1 - 0.2], research paper has taken it as 0.85
# tolerance = 0.1
# L = lexRank(testFile,threshold, tolerance, totalDocs, termDocFrequencyDict)

In [ ]:
# Questions are
# 1) threshold value
# 2) how will I convert L returned by lex rank to readable summary

# Testing the code on first 5 files of cnn and dailymail

In [157]:
testDocSentencesDict, testDocSummaryDict = readCorpus(['Test_cnn', 'Test_dailymail'])
# testDocSentencesDict['dailymail_0a0a733db965c3fdf9bc2895104a1ef884a3d593.story']

['By',
 'Ryan Gorman',
 'PUBLISHED:',
 '16:25 EST, 8 September 2013',
 '|',
 'UPDATED:',
 '04:49 EST, 9 September 2013',
 'A fan who caused outrage by appearing to taunt an alleged sexual assault victim on live television Saturday has explained his actions.',
 'A sign held up by a University of Michigan student behind the hosts of ESPN’s College Gameday pregame show that said ‘Hi Lizzy Seeberg’ sparked outrage and appeared to taunt a girl who accused a University of Notre Dame football player of sexual assaulting her Aug 31, 2010 only to have the allegations appear to have been ignored every step of the way.',
 'Ms Seeberg, 19, was a freshman at St Mary’s College, a small women’s only Catholic college literally across the street from the mighty Notre Dame.',
 "I did it: The person who held up the 'Hi Lizzy Seeberg' sign wrote a letter to Deadspin detailing his reasons behind the offensive gesture",
 'Unable to live with the aftermath of the alleged sexual assault and Notre Dame’s refus

In [156]:
testDocSummaryDict

['Despite widespread outrage, he claims the sign was meant to bring attention to her case, and that of others whose accusations of sexual assault at the University of Notre Dame have been ignored or covered up',
 'Lizzy Seeberg, 19, was a freshman at a small Catholic college across the street from Notre Dame',
 'Ms Seeberg had a history of anxiety and depression, and overdosed on her prescribed medication to kill herself',
 'Notre Dame administration refused for over two years to even acknowledge the accusations, outside of A JOKE made by football coach Brian Kelly',
 'The football player accused of the assault was never suspended, never missed a game and was allowed to attend all practices',
 "The trip was Notre Dame's last to Michigan, the student felt the need to 'stand up for what's right'"]

In [158]:
testRemovedStopWordsDocSentencesDict = removeStopWords(testDocSentencesDict)
# testRemovedStopWordsDocSentencesDict

In [159]:
testStemmedDocSentencesDict, testStemmedWordFrequencyDict = getStemmedText(testRemovedStopWordsDocSentencesDict)
# testStemmedDocSentencesDict

In [160]:
testTermDocFrequencyDict = getTermDocfrequency(testStemmedDocSentencesDict)
# for i in testTermDocFrequencyDict:
#     if testTermDocFrequencyDict[i] == 0:
#         print(i) 

In [161]:
testTotalDocs = len(testDocSentencesDict)
# testTotalDocs

In [163]:
testFile = testStemmedDocSentencesDict['dailymail_0a0a733db965c3fdf9bc2895104a1ef884a3d593.story']
# testFile

In [174]:
#  threshold is in range of [0.1 - 0.3]
#  very high thresholds may lose almost all of the information in a similarity matrix
threshold = 0.1
#  tolerance is also consider as damping factor, range [0.1 - 0.2], research paper has taken it as 0.85
tolerance = 0.1
L = lexRank(testFile,threshold, tolerance, testTotalDocs, testTermDocFrequencyDict)
print(L)

[[ 0.00223214  0.00223214  0.00223214 ...,  0.00223214  0.00223214
   0.00223214]
 [ 0.00223214  0.00223214  0.00223214 ...,  0.00223214  0.00223214
   0.00223214]
 [ 0.00223214  0.00223214  0.00223214 ...,  0.00223214  0.00223214
   0.00223214]
 ..., 
 [ 0.00266344  0.00266344  0.00266344 ...,  0.00266344  0.00266344
   0.00266344]
 [ 0.02237334  0.02237334  0.02237334 ...,  0.02237334  0.02237334
   0.02237334]
 [ 0.01532171  0.01532171  0.01532171 ...,  0.01532171  0.01532171
   0.01532171]]
